In [1]:
!pip install mujoco numpy opencv-python

   ---------------------------------------- 0.0/5.4 MB ? eta -:--:--
   ---------------------------------------- 5.4/5.4 MB 40.8 MB/s  0:00:00
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -------- ------------------------------- 8.4/39.0 MB 40.0 MB/s eta 0:00:01
   ------------ --------------------------- 12.6/39.0 MB 30.3 MB/s eta 0:00:01
   --------------- ------------------------ 15.2/39.0 MB 23.9 MB/s eta 0:00:01
   ------------------ --------------------- 18.4/39.0 MB 22.3 MB/s eta 0:00:01
   ---------------------- ----------------- 21.8/39.0 MB 20.5 MB/s eta 0:00:01
   -------------------------- ------------- 25.4/39.0 MB 20.1 MB/s eta 0:00:01
   ----------------------------- ---------- 29.1/39.0 MB 19.6 MB/s eta 0:00:01
   ---------------------------------- ----- 33.3/39.0 MB 19.8 MB/s eta 0:00:01
   ---------------------------------------  38.5/39.0 MB 20.4 MB/s eta 0:00:01
   ---------------------------------------- 39.0/39.0 MB 19.5 MB/s  0:00:01
 

  You can safely remove it manually.
  You can safely remove it manually.


In [2]:
import mujoco
import numpy as np
import cv2

# Define MuJoCo XML model for a simple 3-DOF robotic arm
xml = """
<mujoco model="robotic_arm">
  <option timestep="0.01" gravity="0 0 -9.81"/>

  <asset>
    <texture name="grid" type="2d" builtin="checker" width="512" height="512"
             rgb1=".1 .2 .3" rgb2=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="1 1" texuniform="true" reflectance=".2"/>
  </asset>

  <worldbody>
    <light pos="0 0 3" dir="0 0 -1" diffuse="1 1 1"/>
    <geom name="floor" type="plane" size="2 2 .1" material="grid"/>

    <!-- Base -->
    <body name="base" pos="0 0 0.1">
      <geom name="base_geom" type="cylinder" size="0.1 0.05" rgba="0.3 0.3 0.3 1"/>
      <joint name="base_joint" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>

      <!-- Link 1 -->
      <body name="link1" pos="0 0 0.05">
        <geom name="link1_geom" type="capsule" size="0.04 0.25" rgba="0.8 0.2 0.2 1"
              fromto="0 0 0 0 0 0.5"/>
        <joint name="shoulder_joint" type="hinge" axis="0 1 0" range="-90 90" damping="0.3"/>

        <!-- Link 2 -->
        <body name="link2" pos="0 0 0.5">
          <geom name="link2_geom" type="capsule" size="0.035 0.2" rgba="0.2 0.8 0.2 1"
                fromto="0 0 0 0 0 0.4"/>
          <joint name="elbow_joint" type="hinge" axis="0 1 0" range="-120 120" damping="0.2"/>

          <!-- End effector -->
          <body name="end_effector" pos="0 0 0.4">
            <geom name="ee_geom" type="sphere" size="0.05" rgba="0.2 0.2 0.8 1"/>
          </body>
        </body>
      </body>
    </body>
  </worldbody>

  <actuator>
    <motor name="base_motor" joint="base_joint" gear="20" ctrllimited="true" ctrlrange="-1 1"/>
    <motor name="shoulder_motor" joint="shoulder_joint" gear="15" ctrllimited="true" ctrlrange="-1 1"/>
    <motor name="elbow_motor" joint="elbow_joint" gear="10" ctrllimited="true" ctrlrange="-1 1"/>
  </actuator>
</mujoco>
"""

# Create model and data
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# Renderer setup
width, height = 640, 480
renderer = mujoco.Renderer(model, height=height, width=width)

# Simulation parameters
duration = 5.0  # seconds
fps = 30
video_filename = "robotic_arm_simulation.mp4"

# Control function - creates a smooth motion
def control_policy(time):
    """Generate control signals for the arm"""
    ctrl = np.zeros(model.nu)
    ctrl[0] = 0.5 * np.sin(2 * np.pi * 0.3 * time)  # Base rotation
    ctrl[1] = 0.3 * np.sin(2 * np.pi * 0.5 * time)  # Shoulder
    ctrl[2] = 0.4 * np.cos(2 * np.pi * 0.4 * time)  # Elbow
    return ctrl

# Setup video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))

# Run simulation with live display and recording
print("Running simulation with live display...")
print("Press 'q' to quit early, or wait for simulation to complete")

time = 0.0
frame_interval = 1.0 / fps
next_frame_time = 0.0

while time < duration:
    # Apply control
    data.ctrl[:] = control_policy(time)

    # Step simulation
    mujoco.mj_step(model, data)

    # Render and display frame at the desired fps
    if time >= next_frame_time:
        renderer.update_scene(data)
        pixels = renderer.render()

        # Convert RGB to BGR for OpenCV
        frame_bgr = cv2.cvtColor(pixels, cv2.COLOR_RGB2BGR)

        # Write to video file
        video_writer.write(frame_bgr)

        # Display live
        cv2.imshow('MuJoCo Robotic Arm Simulation', frame_bgr)

        # Check for quit key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Simulation interrupted by user")
            break

        next_frame_time += frame_interval

    time += model.opt.timestep

# Cleanup
video_writer.release()
cv2.destroyAllWindows()

print(f"\nVideo saved to: {video_filename}")
print("Playing video...")

# Play the saved video
cap = cv2.VideoCapture(video_filename)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow('Recorded Simulation', frame)

    # Play at correct speed
    if cv2.waitKey(int(1000/fps)) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("Done!")

Running simulation with live display...
Press 'q' to quit early, or wait for simulation to complete

Video saved to: robotic_arm_simulation.mp4
Playing video...
Done!
